# EDA Datario
Ao invés de começar por diversas queries dentro do GCP, foi decidido explorar os dados primeiramente através do Pandas, visando decidir todas as variáveis e features necessárias para responder todas as questões do desafio e também ter insights sobre possíveis análises extras.
<br>Objetivos principais:
- criar arquivos .csv para não consumir dados do GCP a cada vez que o código for rodado
- ter uma visão geral de cada dataset
- Decidir quais colunas são necessárias para a soluções de **Todas** as questões do desafio
- Analisar a possibilidade de análises extras relevantes para os dashboards PowerBI e Streamlit.

## Perguntas SQL

## Localização de chamados do 1746
#### Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.

1. Quantos chamados foram abertos no dia 01/04/2023?
2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?


## Chamados do 1746 em grandes eventos
#### Utilize a tabela de Chamados do 1746 e a tabela de Ocupação Hoteleira em Grandes Eventos no Rio para as perguntas de 6-10. Para todas as perguntas considere o subtipo de chamado "Perturbação do sossego".

6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
8. Quantos chamados desse subtipo foram abertos em cada evento?
9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

##### Importante: a tabela de Chamados do 1746 possui mais de 10M de linhas. Evite fazer consultas exploratórias na tabela sem um filtro ou limite de linhas para economizar sua cota no BigQuery!

## Imports

In [1]:
import basedosdados as bd
import pandas as pd
import plotly.express as px
from datetime import timedelta
from utils import Weather   

In [2]:
# Config
wrio = Weather(22.5423, 43.1021) # Cria instância da classe Weather com as coordenadas da cidade do Rio de Janeiro

## Datasets
Conforme primeiras observações feitas no notebook 'api_first_test.ipynb'. Vamos começar a analisar as tabelas do BigQuery em ordem da menor para a maior.
<br>
- datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos
- datario.dados_mestres.bairro
- datario.adm_central_atendimento_1746.chamado

### Fluxo de visitantes/Ocupação de eventos 

Os códigos abaixo estão comentados para evitar o consumo desnecessário de dados no GCP

In [7]:
# Rodar a query apenas uma vez e salvar o dataframe como csv

#query_ocup = "SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos"
#ocup = bd.read_sql(query_ocup, billing_project_id="dados-rio-433111")
#ocup.to_csv('ocupacao.csv', index=False)

Downloading: 100%|██████████|


',ano,data_inicial,data_final,evento,taxa_ocupacao\r\n0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554\r\n1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251\r\n2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184\r\n3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451\r\n'

In [3]:
# Dataset Ocupação Original
ocup = pd.read_csv('ocupacao.csv')
ocup

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


Podemos observar que as observações foram feitas dentro do curto período de setembro/2022 até fevereiro/2023.
<br>Todos os eventos aconteceram em pelo menos 3 dias
<br>O Rock in Rio aconteceu por dois finais de semana seguidos, o ultimo tendo um fluxo de visitantes relativamente maior que o primeiro
<br>O evento de maior fluxo foi o Carnaval, provavelmente por conta de ser feriado público e possívelmente ter sido em dias de calor(verificar com api)

In [4]:
# Info do dataframe original
ocup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ano            4 non-null      object 
 1   data_inicial   4 non-null      object 
 2   data_final     4 non-null      object 
 3   evento         4 non-null      object 
 4   taxa_ocupacao  4 non-null      float64
dtypes: float64(1), object(4)
memory usage: 292.0+ bytes


**Tratativas**

In [10]:
# Converter colunas de datas para datetime
ocup['data_inicial'] = pd.to_datetime(ocup['data_inicial'])
ocup['data_final'] = pd.to_datetime(ocup['data_final'])
# Extrair apenas o ano(s) da coluna ano
ocup['ano'] = ocup['ano'].apply(lambda x: x.split()[-1])
# Criar colunas para o dia da semana de início e fim dos eventos escrita em Português
dias = {
    0: "Segunda-feira",
    1: "Terça-feira",
    2: "Quarta-feira",
    3: "Quinta-feira",
    4: "Sexta-feira",
    5: "Sábado",
    6: "Domingo"
}
ocup['dia_inicial'] = ocup['data_inicial'].apply(lambda x: x.weekday()).map(dias)
ocup['dia_final'] = ocup['data_final'].apply(lambda x: x.weekday()).map(dias)
# Coluna de duração dos eventos(quantidade de dias)
ocup['duracao'] = (ocup['data_final'] - ocup['data_inicial']) + timedelta(days=1)
# Temperatura/clima do dia utilizando o método forecast da classe Weather
ocup['temperatura_data_inicial'] = ocup['data_inicial'].apply(lambda x: wrio.forecast(x.date(), x.date())['temperatura media'])
ocup['clima_data_inicial'] = ocup['data_inicial'].apply(lambda x: wrio.forecast(x.date(), x.date())['clima'])
ocup['temperatura_data_final'] = ocup['data_final'].apply(lambda x: wrio.forecast(x.date(), x.date())['temperatura media'])
ocup['clima_data_final'] = ocup['data_final'].apply(lambda x: wrio.forecast(x.date(), x.date())['clima'])
ocup

,ano,data_inicial,data_final,evento,taxa_ocupacao,dia_inicial,dia_final,duracao,temperatura_data_inicial,clima_data_inicial,temperatura_data_final,clima_data_final
0,2023,2023-02-18,2023-02-21,Carnaval,0.9554,Sábado,Terça-feira,4 days,15.151581,Mainly Sunny,20.324499,Mainly Sunny
1,(2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251,Sexta-feira,Domingo,3 days,10.391168,Rain,13.837001,Light Rain
2,2022,2022-09-02,2022-09-04,Rock in Rio,0.8184,Sexta-feira,Domingo,3 days,30.980749,Sunny,32.174496,Sunny
3,2022,2022-09-08,2022-09-11,Rock in Rio,0.9451,Quinta-feira,Domingo,4 days,32.207832,Sunny,32.826580,Mainly Sunny


**Conclusão**
<br>Apesar de ter sido em época de chuva e temperaturas baixas, o Reveillon teve uma taxa de ocupação mais alta do que no primeiro final de semana do evento Rock in Rio
<br>O carnaval liderou a taxa de ocupação mesmo não sendo um dia considerado "Quente" para o cidadão Carioca

**Sugestão de análise futura**
A taxa de ocupação tem alguma correlação com a média de chamados de Perturbação do Sossego nas datas dos eventos?

### Bairros da cidade de Rio de Janeiro

Os códigos abaixo estão comentados para evitar o consumo desnecessário de dados no GCP

In [11]:
# Rodar a query apenas uma vez e salvar o dataframe como csv

#query_bairro = "SELECT * FROM datario.dados_mestres.bairro"
#bairro = bd.read_sql(query_bairro, billing_project_id="dados-rio-433111")
#bairro.to_csv('bairro.csv', index=False)

Downloading: 100%|██████████|


In [2]:
bairro = pd.read_csv('bairro.csv')

Colunas:
- id_bairro | Código do bairro dado pela prefeitura do Rio de Janeiro.
- nome | Nome do bairro
- id_area_planejamento | Código da área de planejamento onde está localizado a região de planejamento do bairro. Ver atributo nome_regiao_planejamento.
- id_regiao_planejamento | Código da região de planejamento onde está localizado o região administrativa do bairro. Ver atributo REGIAO_ADM.
- nome_regiao_planejamento | Nome da região de planejamento. Deve respeitar o atributo id_regiao_planejamento.
- id_regiao_administrativa | Código da região administrativa onde está locallizado o bairro.
- nome_regiao_administrativa | Nome da região administrativa onde está localizado o bairro. Este atributo deve respeitar o atributo id_regiao_administrativa.
- subprefeitura |Nome da subprefeitura a que pertence o bairro.
- area | Área (m²) do bairro. Tem o mesmo valor do atributo SHAPE.STArea().
- perimetro | Perímetro (m) do bairro.
- geometry_wkt | -
- geometry | Geometria do bairro no banco do ArcGIS.

In [3]:
bairro.head()

,id_bairro,nome,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry
0,2,Gamboa,1,1.1,Centro,1,Portuaria,Centro,1.112903e+06,4612.833630,POLYGON ((-43.18791509600138 -22.8931217212322...,"POLYGON((-43.1879150960014 -22.8931217212322, ..."
1,1,Saúde,1,1.1,Centro,1,Portuaria,Centro,3.638176e+05,2646.220568,POLYGON ((-43.181151633502964 -22.895430284304...,"POLYGON((-43.181151633503 -22.8954302843042, -..."
2,4,Caju,1,1.1,Centro,1,Portuaria,Centro,5.347481e+06,19800.522524,MULTIPOLYGON (((-43.22522241788469 -22.8746498...,MULTIPOLYGON(((-43.196711909178 -22.8872495021...
3,3,Santo Cristo,1,1.1,Centro,1,Portuaria,Centro,1.684721e+06,6743.227885,POLYGON ((-43.194498082949806 -22.903378003392...,"POLYGON((-43.1944980829498 -22.9033780033923, ..."
4,161,Lapa,1,1.1,Centro,2,Centro,Centro,2.983258e+05,3849.181818,POLYGON ((-43.18166120770202 -22.9120798224676...,"POLYGON((-43.181661207702 -22.9120798224677, -..."


In [4]:
bairro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id_bairro                   164 non-null    int64  
 1   nome                        164 non-null    object 
 2   id_area_planejamento        164 non-null    int64  
 3   id_regiao_planejamento      164 non-null    float64
 4   nome_regiao_planejamento    164 non-null    object 
 5   id_regiao_administrativa    164 non-null    int64  
 6   nome_regiao_administrativa  164 non-null    object 
 7   subprefeitura               164 non-null    object 
 8   area                        164 non-null    float64
 9   perimetro                   164 non-null    float64
 10  geometry_wkt                164 non-null    object 
 11  geometry                    164 non-null    object 
dtypes: float64(3), int64(3), object(6)
memory usage: 15.5+ KB


Podemos observar que o dataset original possui 12 colunas e 164 linhas, não possui nenhum valor ausente e os tipos das Series variam entre integers, floats e strings.
<br>As colunas geometry_wkt e geometry possuem coordenadas, mas precisam ser tratadas para que possam ser usadas.
<br>Talvez seja possível plotar as áreas de cada bairro em um mapa

**Subprefeituras e suas cidades**

In [5]:
# Quantas subprefeituras únicas:
print(f'Subprefeituras: {bairro['subprefeitura'].nunique()}')
print(bairro['subprefeitura'].value_counts())

Subprefeituras: 9
subprefeitura
Zona Norte         65
Zona Sul           17
Centro             16
Ilhas              16
Zona Oeste         12
Grande Bangu       12
Jacarepaguá        11
Barra da Tijuca     8
Grande Tijuca       7
Name: count, dtype: int64


**Regiões Administrativas**

In [16]:
print(f'Regiões Administrativas: {bairro['nome_regiao_administrativa'].nunique()}')
print(bairro['nome_regiao_administrativa'].value_counts())

Regiões Administrativas: 33
nome_regiao_administrativa
Meier                 16
Ilha Do Governador    15
Madureira             13
Jacarepagua           10
Barra Da Tijuca        8
Botafogo               8
Lagoa                  7
Inhauma                6
Iraja                  6
Bangu                  6
Pavuna                 6
Realengo               6
Campo Grande           5
Portuaria              4
Vila Isabel            4
Rio Comprido           4
Ramos                  4
Sao Cristovao          4
Guaratiba              4
Vigario Geral          4
Anchieta               4
Santa Cruz             3
Penha                  3
Tijuca                 3
Copacabana             2
Centro                 2
Jacarezinho            1
Rocinha                1
Santa Tereza           1
Paqueta                1
Complexo Da Mare       1
Complexo Do Alemão     1
Cidade De Deus         1
Name: count, dtype: int64


**Região Planejamento**

In [36]:
print(f'Regiões Planejamento: {bairro['nome_regiao_planejamento'].nunique()}')
print(bairro['nome_regiao_planejamento'].value_counts())

Regiões Planejamento: 16
nome_regiao_planejamento
Madureira             19
Zona Sul              18
Méier                 17
Centro                16
Ilha do Governador    15
Bangu                 12
Jacarepaguá           11
Pavuna                10
Barra da Tijuca        8
Penha                  7
Tijuca                 7
Inhaúma                7
Ramos                  5
Campo Grande           5
Guaratiba              4
Santa Cruz             3
Name: count, dtype: int64


In [42]:
#bairro.groupby(['nome_regiao_administrativa', 'nome_regiao_planejamento', 'subprefeitura']).agg({'nome':'count'})
distribuicao = bairro.groupby(['subprefeitura','nome_regiao_planejamento','nome_regiao_administrativa']).agg({'nome':'count'})

distribuicao

nome
subprefeitura   nome_regiao_planejamento nome_regiao_administrativa      
Barra da Tijuca Barra da Tijuca          Barra Da Tijuca                8
Centro          Centro                   Centro                         2
                                         Portuaria                      4
                                         Rio Comprido                   4
                                         Santa Tereza                   1
                                         Sao Cristovao                  4
                Zona Sul                 Botafogo                       1
Grande Bangu    Bangu                    Bangu                          6
                                         Realengo                       6
Grande Tijuca   Tijuca                   Tijuca                         3
                                         Vila Isabel                    4
Ilhas           Centro                   Paqueta                        1
                Ilha do Governador       Ilha Do Governador            15
Jacarepaguá     Jacarepaguá              Cidade De Deus                 1
                                         Jacarepagua                   10
Zona Norte      Inhaúma                  Complexo Do Alemão             1
                                         Inhauma                        6
                Madureira                Iraja                          6
                                         Madureira                     13
                Méier                    Jacarezinho                    1
                                         Meier                         16
                Pavuna                   Anchieta                       4
                                         Pavuna                         6
                Penha                    Penha                          3
                                         Vigario Geral                  4
                Ramos                    Complexo Da Mare               1
                                         Ramos                          4
Zona Oeste      Campo Grande             Campo Grande                   5
                Guaratiba                Guaratiba                      4
                Santa Cruz               Santa Cruz                     3
Zona Sul        Zona Sul                 Botafogo                       7
                                         Copacabana                     2
                                         Lagoa                          7
                                         Rocinha                        1

#### Distribuição Geográfica e Tamanho:

**Quais são os bairros com as maiores e menores áreas?**


In [6]:
bairro[bairro['area'] == bairro['area'].min()][['nome', 'subprefeitura', 'area']]

,nome,subprefeitura,area
114,Zumbi,Ilhas,161117.78418


In [7]:
bairro[bairro['area'] == bairro['area'].max()][['nome', 'subprefeitura', 'area']]

,nome,subprefeitura,area
162,Guaratiba,Zona Oeste,1.317866e+08


**Como as áreas dos bairros estão distribuídas? Existe uma correlação entre área e perímetro?**

In [29]:
fig = px.strip(bairro,y="area", hover_data=['nome', 'area','subprefeitura', 'nome_regiao_administrativa'])#, color='subprefeitura')
fig.show()

A maior parte dos bairros tem a área abaixo de 20M²

In [15]:
fig = px.histogram(bairro, x='nome', y='area')
fig.show()

In [25]:
ar_per = bairro[['area','perimetro']].corr()
ar_per

,area,perimetro
area,1.000000,0.920674
perimetro,0.920674,1.000000


**Qual é a distribuição dos bairros entre as diferentes regiões administrativas?**

In [45]:
qt_bairroRegiao = bairro.groupby('nome_regiao_administrativa').agg({'nome':'count'}).reset_index()

fig = px.histogram(qt_bairroRegiao, x='nome_regiao_administrativa', y='nome', title='Bairros por Região Administrativa')
fig.show()



In [44]:
qt_bairroSubprefeitura = bairro.groupby('subprefeitura').agg({'nome':'count'}).reset_index()

fig = px.histogram(qt_bairroSubprefeitura, x='subprefeitura', y='nome', title='Bairros por Subprefeitura')
fig.show()


In [43]:
qt_bairroPlanejamento = bairro.groupby('nome_regiao_planejamento').agg({'nome':'count'}).reset_index()

fig = px.histogram(qt_bairroPlanejamento, x='nome_regiao_planejamento', y='nome', title='Bairros por Região de Planejamento')
fig.show()


**Plots Geográficos com Geopandas**